<center>
<H1> Calling others languages in *Julia*  <H1>
<img src="julia.png" width="400"
</center> 

# Outline
## calling C code
## calling Fortran code 
## calling Python

# Calling  C code 

 the basic julia call to interface with C code is *ccall* 
 

In [4]:
?ccall

search: ccall AbstractChannel



```
ccall((function_name, library), returntype, (argtype1, ...), argvalue1, ...)
ccall(function_name, returntype, (argtype1, ...), argvalue1, ...)
ccall(function_pointer, returntype, (argtype1, ...), argvalue1, ...)
```

Call a function in a C-exported shared library, specified by the tuple `(function_name, library)`, where each component is either a string or symbol. Instead of specifying a library, one can also use a `function_name` symbol or string, which is resolved in the current process. Alternatively, `ccall` may also be used to call a function pointer `function_pointer`, such as one returned by `dlsym`.

Note that the argument type tuple must be a literal tuple, and not a tuple-valued variable or expression.

Each `argvalue` to the `ccall` will be converted to the corresponding `argtype`, by automatic insertion of calls to `unsafe_convert(argtype, cconvert(argtype, argvalue))`. (See also the documentation for [`unsafe_convert`](@ref Base.unsafe_convert) and [`cconvert`](@ref Base.cconvert) for further details.) In most cases, this simply results in a call to `convert(argtype, argvalue)`.


# Basic example 


In [1]:
io = open("skel.c","w")
write(io, "int ajoute2(int x) { return x+2; }")
close(io)
run(`gcc -o ajoute2.so --shared skel.c`)
w = ccall((:ajoute2, "./ajoute2.so"), Int32, (Int32,), 12)
run(`rm ajoute2.so skel.c`)
println("w = $w")


w = 14


the called function `ajoute2` takes just one argument and returns one argument

# Example with a matrix
- We can call the following C code
```C
#include <inttypes.h>
double mysum(int64_t m, int64_t n, double *x)
{
   double s = 0.;
   int i,j;
   for(i = 0; i < m; ++i)
       for(j = 0; j < n; ++j)
            s += x[i * n + j];
   return s;
}
```
- using 
```julia
a = reshape(collect(1.:10.), 5, 2)
s = ccall((:mysum, "./mysum.so"), Float64, (Int64, Int64, Ptr{Float64}), size(a, 1), size(a, 2), a)
```
## Notes : 
- we use `Ptr{Float64}` to pass a matrix to the function
- using standards types like `int64_t` is not mandatory, but enforces compatibility trough more portability

# Basic C++ support
- we can use C++ code, as long as we provided a `extern "C"` interface for the entry function 

```C
#include <cinttypes>
#include <vector>
#include <numeric>

extern "C" {
double mysum2(int64_t n, double *x);
};

double mysum2(int64_t n, double *x)
{
    double s = 0;
    std::vector<double> t(x, x + n);
    for(auto & z : t) s+= z;
    return s;
}
```
can be called as 

```julia
a = collect(1.:10.)
s = ccall((:mysum2, "./libmysum2.so"), Float64, (Int64, Ptr{Float64}), size(a, 1), a)
println("s = $s")
```

In [2]:
include("C/test_mysum2.jl")

LoadError: error compiling top-level scope: could not load library "./libmysum2.so"
./libmysum2.so: cannot open shared object file: No such file or directory